In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Minimax Search

### Dependencies

In [ ]:
import chess
import random
from typing import Union

import import_ipynb
import Util
from Globals import *

The Minimax Algorithm is a simple algorithm for playing a game.
This algorithm is based on the notion of the value of a State s. In the following, the formal definition of the Minimax-Algorithm, according to Prof. Dr. Karl Stroetmann in his lecture "Artificial Intelligence", is given. Therefore, the source for the following equations is the lecture of Prof. Dr. Karl Stroetmann (Stroetmann, K. 2022)

$State$ is the set of all possible game-states.

Firstly, let us define the function $finished$, which maps a State s to a value of the set $B$, with $B$ beeing defined as $B = \{True, False\} $. Therefore, $finished$ is defined as $$ finished : State \rightarrow B$$
With $finished$ defined, we can now define the set of $TerminalStates$, which is defined as follows:
$$ TerminalStates := \{s \in States | finished(s) \} $$

Secondly, let us define the function utility, which maps a State s to a value. In Prof. Dr. Stroetmanns lecture, this is definded as follows:
$$ utility : TerminalStates \rightarrow \{-1, 0 +1\} $$
Note, that the number of possible values in a chess game is much greater than defined in the lecture for this particular case. More in the following. 

Now let us define the function $value$. $Value$ is conceptionally an extension to the notion of $utility$ of a state. However, instead of mapping a value to a $TerminalState$, the function $value$ is defined for all states. Formally, we define a function 
$$maxValue : State \rightarrow  \{-1, 0 +1\} $$
This function takes a state s and returns a value that state has for the first player, who tries to maximize the value of the state. Here, we assume that all players play optimally.
This function is defined by recursion.
Due to the $maxValue$ function beeing an extension to the $utility$ function, the base case is as follows:
$$ finished(s) \rightarrow maxValue(s) = utility(s)$$
If the game is not finished, we define
$$ \neg finished(s) \rightarrow maxValue(s) = max(\{minValue(n) | n  \in nextStates(s,gPlayers[0])\}). $$ 

The reason is that, if a game is not finished yet, the maximizing player gPlayers[0] has to evaluate all possible moves, in order to take the one move, which benefits him the most. Therefore, the player computes the set $nextStates(s, gPlayers[0])$ of all states that can be reached from the current state s. Next is the minimizing player gPlayers[1]. This player tries to minimize the possible value for the enemy player gPlayers[0]. Hence, in order to evaluate the state n, we call the function $minValue$ recursively as $minValue(n)$.
The function $minValue$ is defined by the following recursive equations and has the same signature as $maxValue$:  

$1.   finished(s) \rightarrow minValue(s) = utility(s).$  
$2.   \neg finished(s) \rightarrow minValue(s) = min(\{maxValue(n) | n \in nextStates(s, gPlayers[1])\}).$

In the following, we will speak of the $value$ function, which is used as a synonym for the function $maxValue$


In the case of this implementation, the set of values is much greater. In this case, value maps the current board-state onto a number, calculated by taking into account the different piece-values and piece-square-tables. Therefore, value can map onto a range of -100 000 000 to 10 000 000, which is defined in Globals.py.  
Instead of calling another function, we use the negated values of $ai_turn$ and $color$ to call the same function for the other player, hence calling the function recursively to calculate the best move.

Optionally, the algorithm uses memoization, which is a type of caching. This works by mapping a board state, plus iteration details, to the corresponding score and move and storing it in a cache dictionary. If, during a later iteration, the same board state is reached on the same iteration, these values can be read from the cache.

#### chess.Board.get_best_move_minimax
Finds the best move to make based on the minimax algorithm. This algorithm works as follows:
- Iterate over all legal moves in the current position.
- For each move, find the best possible score after making this move.
    - This is done by calling the minimax function recursively: increasing the current iteration by 1 and switching turns.
- Find the maximum (if it's the AI's turn) or the minimum (if it's the other player's/AI's turn) score of all legal moves, alongside the move that was able to reach this optimized state. This is the move that the algorithm recommends.

Optionally, the algorithm uses memoization, which is a type of caching. This works by mapping a board state, plus iteration details, to the corresponding score and move and storing it in a cache dictionary. If, during a later iteration, the same board state is reached on the same iteration, these values can be read from the cache.

__This function is implemented recursively.__

###### <b><u>Arguments</u></b>
``cache (dict):``  
A cache dictionary if memoization is desired, or None if memoization should be disabled.  

``use_heuristic (bool):``  
Whether or not the heuristic for evaluating the chess board should be used. Chess problems don't need this heuristic.  

``ai_turn (bool):``  
Whether or not the current turn is of the AI that started the search. If this is the case, the score should be maximized. Otherwise, the score should be minimized.  

``color (chess.Color):``  
The color of the player whose turn it currently is.  

``endgame_tablebase (Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase]):``  
The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.  

``iteration (int):``  
The depth of the search (amount of moves currently looking ahead).  

``max_iterations (int):``  
The maximum depth of the search.  

``last_eval_score (int):``  
The score provided by the previous evaluation in the search. 

###### <b><u>Returns <i>(int, chess.Move, bool)</i></u></b>
- The board score after making the recommended best move.
- The recommended best move to make.
- Whether or not the endgame library was used to find the move.

###### <b><u>Side effects</u></b>
- If a cache dictionary is provided, new values may be added to this dictionary.
- If the search is interrupted, the board may be in a different state than when the search started.

In [ ]:
def get_best_move_minimax(
    self,
    cache: dict,
    use_heuristic: bool,
    ai_turn: bool,
    color: chess.Color,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase,
            chess.gaviota.PythonTablebase],
    iteration: int,
    max_iterations: int,
    last_eval_score: int
) -> (int, chess.Move, bool):

    if cache is not None and (iteration, self.get_state_string()) in cache:
        return cache[(iteration, self.get_state_string())]

    original_color = color if ai_turn else not color
    result_score = self.get_search_result_if_finished(original_color, iteration,
            max_iterations, last_eval_score)
    if result_score is not None:
        return result_score, None, False

    # Check additional moves using the minimax algorithm
    best_score = Globals.MAX_EVALUATION_SCORE * (-1 if ai_turn else 1)
    best_move = None
    best_move_used_endgame = False
    for move in self.legal_moves:
        eval_score, used_endgame_anywhere = self.evaluate_move(
                use_heuristic, color, not ai_turn, last_eval_score,
                iteration, move, endgame_tablebase)
        self.push(move)
        
        if self.is_game_over():
            score_after_move = eval_score
        else:
            score_after_move, _, used_endgame = self.get_best_move_minimax(
                    cache, use_heuristic, not ai_turn, not color,
                    endgame_tablebase, iteration + 1, max_iterations, eval_score)
            used_endgame_anywhere = used_endgame_anywhere or used_endgame

        self.pop()

        if (ai_turn and score_after_move > best_score) \
                or (not ai_turn and score_after_move < best_score):
            best_score = score_after_move
            best_move = move
            best_move_used_endgame = used_endgame_anywhere

    if cache is not None:
        cache_key = (iteration, self.get_state_string())
        cache[cache_key] = best_score, best_move, best_move_used_endgame

    return best_score, best_move, best_move_used_endgame

chess.Board.get_best_move_minimax = get_best_move_minimax
del get_best_move_minimax

#### make_move_minimax
Finds the best possible move according to the minimax algorithm and pushes it onto the move stack.

###### <b><u>Arguments</u></b>
``board (chess.Board):``  
The board to push the move to.  

``color (chess.Color):``  
The color of the player that makes the move.  

``search_depth (int):``  
The iteration depth of the minimax search.  

``endgame_tablebase (Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase]):``  
The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.  

``use_heuristic (bool):``  
Whether or not the heuristic for evaluating the chess board should be used. Chess problems don't need this heuristic.

###### <b><u>Returns <i>(int, chess.Move, bool)</i></u></b>
- The evaluated score of the best possible move.
- The best possible move that was found.
- Whether or not the endgame library was used to find the move.

###### <b><u>Side effects</u></b>
- The best possible move is pushed to the move stack of the board.

In [ ]:
def make_move_minimax(
    board: chess.Board,
    color: chess.Color,
    search_depth: int,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase,
            chess.gaviota.PythonTablebase],
    use_heuristic: bool
) -> (int, chess.Move, bool):

    score, move, used_endgame = board.get_best_move_minimax(
        cache=None, 
        use_heuristic=use_heuristic,
        ai_turn=True, 
        color=color, 
        endgame_tablebase=endgame_tablebase,
        iteration=0, 
        max_iterations=search_depth, 
        last_eval_score=0 # The starting board score doesn't matter,
                          # it's evaluated by score difference
    )
    board.push(move)
    return score, move, used_endgame

#### make_move_minimax_memoization
Finds the best possible move according to the minimax algorithm and pushes it onto the move stack. The algorithm is optimized using memoization (a type of caching).

###### <b><u>Arguments</u></b>
``board (chess.Board):``  
The board to push the move to.  

``color (chess.Color):``  
The color of the player that makes the move.  

``search_depth (int):``  
The iteration depth of the minimax search.  

``endgame_tablebase (Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase]):``  
The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.  

``use_heuristic (bool):``  
Whether or not the heuristic for evaluating the chess board should be used. Chess problems don't need this heuristic.

###### <b><u>Returns <i>(int, chess.Move, bool)</i></u></b>
- The evaluated score of the best possible move.
- The best possible move that was found.
- Whether or not the endgame library was used to find the move.

###### <b><u>Side effects</u></b>
- The best possible move is pushed to the move stack of the board.

In [ ]:
def make_move_minimax_memoization(
    board: chess.Board,
    color: chess.Color,
    search_depth: int,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase,
            chess.gaviota.PythonTablebase],
    use_heuristic: bool,
) -> (int, chess.Move, bool):

    score, move, used_endgame = board.get_best_move_minimax(
        cache={}, 
        use_heuristic=use_heuristic,
        ai_turn=True, 
        color=color, 
        endgame_tablebase=endgame_tablebase,
        iteration=0, 
        max_iterations=search_depth, 
        last_eval_score=0 # The starting board score doesn't matter,
                          # it's evaluated by score difference
    )
    board.push(move)
    return score, move, used_endgame

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>